In [1]:
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

2.4.0


## バッチ正規化

```python
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])
```

In [2]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1

In [4]:
# 訓練可能か確認
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [5]:
model.layers[1].updates

/opt/anaconda3/envs/local/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer.py:1402: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


[]

In [7]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal", use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal", use_bias=False),
    keras.layers.Activation("elu"),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    keras.layers.Dense(10, activation="softmax")
])

# 転移学習

```python
model_A = keras.models.load_model("my_model_A.h5")   # モデルをロード
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

# model_Aとmodel_B_on_Aが同じ層を共有することを避ける。
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

# 新しい出力層は無作為に初期化されているため、大きな誤差を生んでしまう。
# この誤差により大きな誤差勾配が生まれ、再利用した重みが壊れてしまう。
# 対策として、最初の数エポックのうちは再利用した層を凍結し、新しい層に妥当な重みを学習するための時間を与える。
# 層を凍結したり、凍結解除した時には、必ずモデルをコンパイルしなければいけない。
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])

# モデルを数エポック訓練しれから、再利用の層の凍結を解除して訓練を続け、再利用の層をタスクBに合わせて微調整する。
# 再利用の像の凍結を解除した後は、それらの層の重みを壊さないように学習率を下げる。
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B,y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True
    
optimizer = keras.optimizer.SGD(lr=1e-4)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16, validation_data=(X_valid_B,y_valid_B))
```

# 正則化



```python
layer = keras.layers.Dense(100, activation="elu",
                           kernel_initalizer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01)
                          )
```

- 何度も同じことを書くのではなく、最初に定義してしまう。

```python
from functools import partial

RegulatrizerDense = partial(keras.layers.Dense,
                            activation="elu",
                            kernel_initalizer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01)
                          )

model = keras.layers.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegulatrizerDense(300),
    RegulatrizerDense(100)
    RegulatrizerDense(10, activation="softmax",
                      kernel_initalizer="glorot_uniform"
                     )
])
```

# ドロップアウト

- p=50%にすると、テスト中のニューロンは、訓練中の平均にして２倍の入力ニューロンが接続される。
    - 訓練後は、ここの入力ニューロンの接続重みを0.5倍しなければいけない。
    - これを忘れると、各ニューロンは全体として訓練中の２倍の入力信号を受けることになり、うなく機能しなくなる。
- モデルが過学習しているなら数値を上げ、訓練セットに過少適合しているなら数値を下げる。

# ガイドライン

| ハイパーパラメータ | デフォルト値 |
| :--- | :--- |
| 初期値 | Heの初期値 |
| 活性化関数 | ELU |
| 正規化 | 浅い場合はなし、深い場合はバッチ正規化 |
| 正則化 | 早期打ち切り（さらに必要ならL2正則化） |
| オプティマイザ | もーメンタム最適化 |
| 学収率スケジューリング | １サイクル |